# Post-processing notebook

## Load and calculate variables from output files

Load modules, and set run directory

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import cmocean
%config InlineBackend.figure_format = 'retina'

rundir='../example_run/'

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [8.0, 6.0]
mpl.rcParams['figure.dpi'] = 100

Read variables in from HDF5 files

In [ ]:
import hdf5_read as h5r

Re, Ri_t, Pr = h5r.read_input(rundir)

tii, U1rms, U2rms, U3rms, THrms, THflux, eps_av, chi_av = h5r.read_stat(rundir)

U1me, U3me, THme, epsilon, chi, THflux, U1U2, U3U2, U1U1, U2U2, U3U3, THTH, nk, ny = h5r.read_mean(rundir)

E1, E2, E3, ETH, nky = h5r.read_spectra(rundir)

Calculate more volume-averaged quantities from the stats.h5 variables

In [ ]:
L_K_av=(Re**3*eps_av)**(-1./4)
EK_av=0.5*(U1rms**2+U2rms**2+U3rms**2)
EP_av=0.5*Ri_t*THrms**2
Re_lambda=EK_av*np.sqrt(15*Re/eps_av)
Re_b_av=eps_av*Re/Ri_t
eta_i=chi_av/(chi_av+eps_av)
eta_c=np.zeros(tii.shape)
eta_c[0]=eta_i[0]
for i in range(1,tii.size):
    eta_c[i]=np.trapz(chi_av[:i+1],x=tii[:i+1])/(np.trapz(chi_av[:i+1],x=tii[:i+1])+np.trapz(eps_av[:i+1],x=tii[:i+1]))

Calculate horizontally-averaged quantities from the mean.h5 variables

In [ ]:
# Energy
EKme=0.5*(U1me**2+U3me**2)
EK=0.5*(U1U1**2+U2U2**2+U3U3**2)
EPme=Ri_t/2*THme**2
EP=Ri_t/2*THTH**2

# Shear
CIKY=1j*np.concatenate(
    (np.arange(np.ceil((ny+1)/2)), np.arange(np.ceil(-(ny-1)/2),0.)))
import numpy.fft as ft
CS1=ft.fft(U1me,axis=1)
CS1=CIKY*CS1
DU1DY=np.real(ft.ifft(CS1,axis=1))

CS1=ft.fft(U3me,axis=1)
CS1=CIKY*CS1
DU3DY=np.real(ft.ifft(CS1,axis=1))

S2=DU1DY**2+DU3DY**2
S_p=U1U2*DU1DY+U3U2*DU3DY

# Stratification
CS1=ft.fft(THme,axis=1)
CS1=CIKY*CS1
DTHDY=np.real(ft.ifft(CS1,axis=1))
N2=Ri_t*(1+DTHDY)

# Length scales
L_K=(Re**3*epsilon)**(-1./4)
L_O=epsilon**0.5*N2**-0.75
L_b=U2U2/np.sqrt(N2)
L_E=THTH/N2
L_C=epsilon**0.5*S2**-0.75

Re_b=Re*epsilon/N2
Ri_g=N2/S2
Fr_t=epsilon/np.sqrt(N2)/EK
K_rho=chi/N2**2

Calculate volume-averaged quantities from the vertical profiles

In [ ]:
EK_bar=np.mean(EKme,axis=1)
eps_bar=1/Re*np.mean(S2,axis=1)
L_O_av=np.mean(L_O,axis=1)
Ri_g_av=np.mean(N2,axis=1)/np.mean(S2,axis=1)

## Plotting examples

#### Time series (e.g. dissipation rate)

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=16)

plt.figure(1)
plt.plot(tii, eps_bar+eps_av+chi_av,'k', label='Total')
plt.plot(tii, eps_bar,'r', label='$\overline{\epsilon}$')
plt.plot(tii, eps_av,'g', label='$\epsilon$')
plt.plot(tii, chi_av,'b', label='$\chi$')

plt.xlabel('$t$')
plt.ylabel('Dissipation rate')
plt.ylim([0, np.max(eps_bar+eps_av+chi_av)])
plt.xlim([tii[0], tii[-1]])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.legend()
#plt.savefig('dissipation.svg',format='svg')
plt.show()

#### Hovm&ouml;ller plot of vertical profiles with time (e.g. vertical diffusivity)

In [ ]:
yvec=np.linspace(2*np.pi/ny,2*np.pi,ny)

plt.figure(2,figsize=(12.0,6.0))
plt.pcolormesh(tii,yvec,K_rho.T, cmap=cmocean.cm.tempo, vmin=0)

c = plt.colorbar()
c.formatter.set_powerlimits((0, 0))
c.update_ticks()

plt.yticks([2*np.pi/ny,np.pi,2*np.pi],('0','$\pi$','$2\pi$'))
plt.ticklabel_format(style='sci', axis='c', scilimits=(0,0))
plt.xlabel('$t$')
plt.ylabel('$z$')
plt.title('Vertical diffusivity $K_\\rho=\chi/(d\\rho/dz)^2$')

plt.show()

#### Vertical wavenumber spectrum at a single time (e.g. kinetic energy spectrum)

In [ ]:
KY=np.arange(nky+1)

plt.figure(3)
idx=100;
plt.loglog(KY,(E1[idx]+E2[idx]+E3[idx]),'k',label='Total')
plt.loglog(KY,E1[idx],'r',label='U1')
plt.loglog(KY,E2[idx],'g',label='U2')
plt.loglog(KY,E3[idx],'b',label='U3')
#plt.loglog(KY[1:11],0.1*KY[1:11]**(-2.),'k--')
#plt.ylim([1e-6, 1e-1])
plt.legend()
plt.title('Kinetic energy spectrum at time $t='+'{:.2f}'.format(tii[i])+'$')
plt.xlabel('$|m|$')
plt.ylabel('$E(|m|)$')

plt.show()

#### Time series for checking energy input rate from forcing

In [ ]:
E=EK_bar+EK_av+EP_av
dEdt=(E[2:]-E[:-2])/(tii[2:]-tii[:-2])
P=dEdt+eps_av[1:-1]+chi_av[1:-1]+eps_bar[1:-1]
Reb_T=10

plt.figure(4)
plt.plot(tii[1:-1], P)
plt.plot(tii, np.ones(np.size(tii))*Reb_T*Ri_t/Re,'k--')
plt.ylim([0, 1.2*Reb_T*Ri_t/Re])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.xlabel('$t$')
plt.ylabel('$dE/dt+\overline{\epsilon}+\epsilon+\chi\\approx P$')

plt.show()

## Write out movie from movie_ab.h5

In [ ]:
import read_movie

plane='xy'
var='TH1'

read_movie.make_movie(rundir,plane,var)

ffmpeg can be used to combine the resulting images (stored in the subdirectory ./tmp/) into a single video, e.g. using the command

ffmpeg -i tmp/%*.png -crf 17 movie.mp4